In [1]:
import calendar
import datetime
import dateutil.parser
import gpxpy 
import gpxpy.gpx
import os
import pandas as pd
import pickle
import pendulum
import pydarksky
import numpy as np
import requests
import time
import timeit

from glob import glob
from xml.dom import minidom
#from gpx_csv_converter import Converter

In [2]:
# Modify gpx_csv_converter CLASS

# #https://ocefpaf.github.io/python4oceanographers/blog/2014/08/18/gpx/

def iso_to_epoch(iso_time):
    return calendar.timegm(dateutil.parser.parse(iso_time).timetuple())


class Converter:

    def __init__(self, string, name):

        if name[-4:] != '.csv':
            name = name + '.csv'

        # parse an xml file by name
        mydoc = minidom.parseString(string)

        trkpt = mydoc.getElementsByTagName('trkpt')
        time = mydoc.getElementsByTagName('time')
        ele = mydoc.getElementsByTagName('ele')
        hr = mydoc.getElementsByTagName('gpxtpx:hr')
        cad = mydoc.getElementsByTagName('gpxtpx:cad')
        temp = mydoc.getElementsByTagName('gpxtpx:atemp')
        
        #hr = mydoc.getElementsByTagName('ns3:hr')
        #cad = mydoc.getElementsByTagName('ns3:cad')

        lats = []
        longs = []
        times = []
        eles = []
        hrs = []
        dates = []
        parsed_times = []
        cads = []
        temps = []

        for elem in trkpt:
            lats.append(elem.attributes['lat'].value)
            longs.append(elem.attributes['lon'].value)

        for elem in time:
            times.append(elem.firstChild.data)

        for elem in hr:
            hrs.append(elem.firstChild.data)

        base_time = iso_to_epoch(times[0])

        time_differences = []

        for item in times:
            time_differences.append(iso_to_epoch(item) - base_time)
            date_obj = (dateutil.parser.parse(item))
            dates.append(str(date_obj.year) + "-" + str(date_obj.month) + "-" + str(date_obj.day))
            parsed_times.append(str(date_obj.hour) + ":" + str(date_obj.minute) + ":" + str(date_obj.second))

        for elem in ele:
            eles.append(elem.firstChild.data)

        for elem in cad:
            cads.append(elem.firstChild.data)
   
        for elem in temp:
            temps.append(elem.firstChild.data)
        
        
        hrs.append(0)

        data = {'date': pd.Series(dates),
                'time': pd.Series(parsed_times),
                'latitude': pd.Series(lats),
                'longitude': pd.Series(longs),
                'elevation': pd.Series(eles),
                'heart_rate': pd.Series(hrs),
                'cadence': pd.Series(cads),
                'temperature': pd.Series(temps)}

        #print(len(dates), len(parsed_times), len(lats), len(longs), len(eles), len(hrs), len(cads), len(temps))

        df = pd.DataFrame(data=data)

        df = df[['date', 'time', 'latitude', 'longitude', 'elevation', 'heart_rate', 'cadence', 'temperature']]

        df.to_csv(name, encoding='utf-8', index=False)

In [3]:
# Path to data - files operations

#PATH_TO_DATA = '~/geekhubds/project/'
PATH_TO_DATA = os.path.abspath(os.curdir)
path_to_files = 'gpx\*'

# File operations
file_quant = len(glob(path_to_files))
file_names = [file for file in glob(path_to_files)]
file_length = len(file_names)

#files_gpx = os.listdir('gpx')
#files_gpx = [('gpx\\' + file) for file in os.listdir('gpx')]
files_csv = [ ('csv\\' + file[:len(file) - 4] + '.csv') for file in os.listdir('gpx')]

print(' path from gpx:\n',file_names)
print('\n path to csv: \n',files_csv)

 path from gpx:
 ['gpx\\Snow_corridor_Street_porridge.gpx', 'gpx\\Snow_waist_Water_under_snow_Hard_snow.gpx', 'gpx\\Some_intervals_in_snow_Frosty_roads.gpx']

 path to csv: 
 ['csv\\Snow_corridor_Street_porridge.csv', 'csv\\Snow_waist_Water_under_snow_Hard_snow.csv', 'csv\\Some_intervals_in_snow_Frosty_roads.csv']


In [4]:
%%time

# OPEN file for XML parse

gpx_file_xml = []

# convert gpx -> csv
for ind, (csv, file) in enumerate(zip(files_csv, file_names)):
    
    start_time = timeit.default_timer()
    
    gpx_file = open( file, 'r' )
    gpx_file_str = gpx_file.read()
    
    # parse gps by xml tags
    Converter(gpx_file_str, csv[:len(csv) - 4])
   
    gpx_file.close()
    
    end_time = timeit.default_timer() - start_time
    
    print(f'file: {ind + 1} -> {file}  time: {end_time:.0f} s \n')

file: 1 -> gpx\Snow_corridor_Street_porridge.gpx  time: 4 s 

file: 2 -> gpx\Snow_waist_Water_under_snow_Hard_snow.gpx  time: 5 s 

file: 3 -> gpx\Some_intervals_in_snow_Frosty_roads.gpx  time: 4 s 

Wall time: 13.4 s


In [5]:
%%time

# OPEN file gpx by libraries

gpx_file_xml = []

for ind, (csv, file) in enumerate(zip(files_csv, file_names)):
    
    start_time = timeit.default_timer()
    
    gpx_file = open( file, 'r' )

    gpx_file_xml_tmp =  gpxpy.parse(gpx_file)
    gpx_file_xml.append(gpx_file_xml_tmp)
    
    gpx_file.close()
    
    end_time = timeit.default_timer() - start_time
    
    print(f'file: {ind + 1} -> {file}  time: {end_time:.0f} s \n')

file: 1 -> gpx\Snow_corridor_Street_porridge.gpx  time: 2 s 

file: 2 -> gpx\Snow_waist_Water_under_snow_Hard_snow.gpx  time: 2 s 

file: 3 -> gpx\Some_intervals_in_snow_Frosty_roads.gpx  time: 2 s 

Wall time: 5.68 s


In [6]:
# Import converted csv files

#user_data = [pd.read_csv(os.path.join('C:\\Users\\philka-ua\\geekhubds\\project\\',file)) for file in files_csv]
user_data = [pd.read_csv(os.path.join(PATH_TO_DATA,file)) for file in files_csv]

# Convert time and date to timestamp format (All files)
[data.insert(2,'data_time',(data['date'] + ' ' + data['time'])\
        .apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ))) for data in user_data]

# # Convert time and date to timestamp format (One file)
# data_time_timestamp = (user_data[0]['date'] + ' ' + user_data[0]['time'])\
#         .apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ))
# user_data[0].insert(2,'data_time',data_time_timestamp)

# #user_data[0]['data_time'] = data_time_timestamp.values.astype('datetime64[s]').astype(datetime.datetime)
# #user_data[0].insert(2,'data_time',data_time_timestamp.values.astype('datetime64[s]').astype(datetime.datetime))

user_data[0].head(10)

,date,time,data_time,latitude,longitude,elevation,heart_rate,cadence,temperature
0,2019-1-28,13:56:10,2019-01-28 13:56:10,49.409331,32.097608,90.4,97,61.0,NaN
1,2019-1-28,13:56:10,2019-01-28 13:56:10,49.409345,32.097593,90.4,98,61.0,NaN
2,2019-1-28,13:56:11,2019-01-28 13:56:11,49.409358,32.097570,90.2,98,61.0,NaN
3,2019-1-28,13:56:12,2019-01-28 13:56:12,49.409369,32.097546,90.2,97,61.0,NaN
4,2019-1-28,13:56:13,2019-01-28 13:56:13,49.409380,32.097521,90.2,97,61.0,NaN
5,2019-1-28,13:56:14,2019-01-28 13:56:14,49.409392,32.097495,89.8,97,85.0,NaN
6,2019-1-28,13:56:15,2019-01-28 13:56:15,49.409405,32.097469,89.8,97,85.0,NaN
7,2019-1-28,13:56:16,2019-01-28 13:56:16,49.409411,32.097439,89.3,98,85.0,NaN
8,2019-1-28,13:56:17,2019-01-28 13:56:17,49.409413,32.097411,89.1,98,84.0,NaN
9,2019-1-28,13:56:18,2019-01-28 13:56:18,49.409405,32.097384,89.3,98,83.0,NaN


In [7]:
%%time

# GPX FAETURES adding (by the imported files with gpx libruary)


# Segments of all gpx track
all_segments = [*map(lambda x: x.tracks[0].segments[0], gpx_file_xml)]

# Point for all gpx track
all_points = [*map(lambda x: x.points, all_segments)]
#[*map(lambda p: p.elevation, map(lambda x: x.points, all_segments) )]


# ELEVATIONS

# Elevation from gpx points
def elevation_get(points):
    '''get evaluation value from GPX point'''
    return map(lambda e: e.elevation, points)

# Elevation for all gpx tracks 
all_elevation = [*map(list, map(elevation_get, all_points))]
#[[*map(lambda e: e.elevation, points)] for points in all_points]

# Grade(Slopes) from gpx points
def grade_get(points, steps=25):
    '''get evaluation slope from GPX point'''
    grd = []
    
    for ind, element in enumerate(points[:-steps]): 
        init_element = points[ind + steps]
        delta_grd = element.elevation_angle(init_element)
        
        grd.append(delta_grd)
    
    grd = grd + [grd[-1]] * steps
    
    return  map(lambda x: round(x, 1), grd)
   
# function for elevation
def uphill_gain(elevation):
    '''get uphill in each point'''
    up = [0]
    init_element = elevation[0]

    for element in elevation: 
        delta_elevation = element - init_element
        init_element = element
        if delta_elevation > 0:
            up.append(up[-1] + delta_elevation)
        else:
            up.append(up[-1])
    return list( map(lambda x: round(x, 5), up[1:]) )

def downhill_loss(elevation):
    '''get downhill in each point'''
    dw = [0]
    init_element = elevation[0]

    for element in elevation: 
        delta_elevation = element - init_element
        init_element = element
        if delta_elevation < 0:
            dw.append(dw[-1] + abs(delta_elevation))
        else:
            dw.append(dw[-1])
    return list( map(lambda x: round(x, 1), dw[1:]) )

def elevation_between(elevation):
    '''get elevetions between each neighbours points from start'''
    el = [0]
    init_element = elevation[0]

    for element in elevation: 
        delta_elevation = element - init_element
        init_element = element
        el.append(delta_elevation)

    return list( map(lambda x: round(x, 5), el[1:]) )


# grade (slope) for a few points
all_grade = [*map(list, map(grade_get, all_points))]

# uphill & downhill in each point
all_uphill_gain = [*map(uphill_gain, all_elevation)]
all_downhill_loss = [*map(downhill_loss, all_elevation)]

# Elevation between each points
all_elevation_between = [*map(elevation_between, all_elevation)]


# SPEED

# Speed in each point
def speed_evaluating(segments):
    '''evaluating speed in each point in km/hour'''
    return map(lambda x: round(x * 3.6, 3), map(segments.get_speed, range(segments.points.__len__())) )

all_speed = [*map(list, map(speed_evaluating, all_segments))]


# DISTANCE

# function for distance
def dist_3d(points):
    '''get distance_3d in each point from start'''
    dst = [0]
    init_element = points[0]

    for element in points: 
        delta_dst = init_element.distance_3d(element)
        dst.append(dst[-1] + delta_dst)
    
    return list( map(lambda x: round(x, 1), dst[1:]) )

def dist_2d(points):
    '''get distance_2d in each point from start'''
    dst = [0]
    init_element = points[0]

    for element in points: 
        delta_dst = init_element.distance_2d(element)
        dst.append(dst[-1] + delta_dst)
    
    return list( map(lambda x: round(x, 1), dst[1:]) )

def dist__2d_between(points):
    '''get distance_2d between each neighbours points from start'''
    
    dst = [0]
    init_element = points[0]

    for element in points: 
        delta_dst = init_element.distance_2d(element)
        init_element = element
        dst.append(delta_dst)
    
    return list( map(lambda x: round(x, 2), dst[1:]) )

# distance evaluating for each point
all_distance_2d = [*map(list, map(dist_2d, all_points))]
all_distance_3d = [*map(list, map(dist_3d, all_points))]

# distance evaluating for each point
all_distance_between_2d = [*map(list, map(dist__2d_between, all_points))]


# Gradient (my function)

def gradient_get(data):
    '''evaluating gradient in each point
    
    The steepness of ups and downs is called a slope. 
    The slope is expressed as a percentage and is determined by the formula:
    grad = (elevation/distance) * 100
    where elevation is the height of ascent or descent;
    distance is the length of the ascent or descent.  
    '''
    if data[1] == 0:
        grad = 0
    else:
        grad = (data[0]/data[1]) * 100
    
    return grad

def gradient_averge_get(data_points, step=50):
    '''
    evaluating avarge gradient for several points
    
    The steepness of ups and downs is called a slope. 
    The slope is expressed as a percentage and is determined by the formula:
    grad = (elevation/distance) * 100
    where elevation is the height of ascent or descent;
    distance is the length of the ascent or descent.
    '''
    elevation = []
    distance = []
    
    for el, dst in data_points:
        elevation.append(el)
        distance.append(dst)
         
    grad_avarge = []
    
    for i in range(len(elevation) - step):
    
        if sum(distance[i:i+step]) != 0:
               grad_avarge.append( sum(elevation[i:i+step]) / sum(distance[i:i+step]) * 100 )
        else:
               grad_avarge.append(0)
        
    return list( map(lambda x: round(x, 1), grad_avarge + [grad_avarge[-1]] * step) )

# Elevations and distance in tuple
elevation_distance = [*map(list, map(lambda x: zip(x[0],x[1]) , zip(all_elevation_between, all_distance_between_2d) ))]
#elevation_distance = [*map(list, [zip(elv,dst) for elv,dst in zip(all_elevation_between, all_distance_between_2d)] )]

# Slope in each point
all_slope_points = [[*map(lambda x: round(x, 1), map(gradient_get, el_dist))] for el_dist in elevation_distance]
# Slope by steps points
all_slope_stp_points = [*map(gradient_averge_get,elevation_distance)]

Wall time: 1.01 s


In [8]:
%%time
# Add GPS fetures to csv files

# # Fetures decribes
#
# speed - speed in each GPX pont
# distance_2d - accumulating distance by GPX point (by lot and lat - parameters)
# distance_3d - accumulating distance by GPX point (by elevation, lot and lat - parameters)
# distance_points_2d - distance between each GPX points ( by lot and lat - parameters))
# uphill_gain - accumulating elevation by GPX points - disribes uphill duaring moving track
# downhill_loss - accumulating lost elevation by GPX points - disribes downhill duaring moving track
# grade / slope - the steepness of ups and downs is called a slope. The slope is expressed as a percentage
#               parameter evaluated for several (steps - parametr) GPX point - window method
#               slope - evaluated by my function, grade - evaluated by bildin function in GPX library
# slope_points - same parametr as grade / slope, but evaluated for each neighboring points


column = ['speed', 'distance_2d', 'distance_3d', 'distance_points_2d', 'uphill_gain',\
          'downhill_loss','grade','slope','slope_points']

user_data_features = []

for sp, d2, d3, d2_bt, up, dw, gr, sl, slp, data in zip(all_speed, all_distance_2d, all_distance_3d,\
    all_distance_between_2d,all_uphill_gain,all_downhill_loss,all_grade,all_slope_stp_points, all_slope_points, user_data):
    # stack parameters
    stacked_array = np.vstack((sp, d2, d3, d2_bt, up, dw, gr, sl, slp)).T
    
    df = pd.DataFrame(stacked_array,columns=column)
    
    # result dataframe
    user_data_features.append( pd.concat([data, df], axis=1) )

Wall time: 32.2 ms


In [9]:
user_data_features[2].head(5)

,date,time,data_time,latitude,longitude,elevation,heart_rate,cadence,temperature,speed,distance_2d,distance_3d,distance_points_2d,uphill_gain,downhill_loss,grade,slope,slope_points
0,2019-1-25,13:4:52,2019-01-25 13:04:52,49.409354,32.097220,88.6,91,63.0,NaN,5.070,0.0,0.0,0.00,0.0,0.0,-0.3,-1.4,0.0
1,2019-1-25,13:4:52,2019-01-25 13:04:52,49.409346,32.097205,88.7,91,62.0,NaN,5.812,1.4,1.4,1.40,0.1,0.0,-0.4,-1.4,7.1
2,2019-1-25,13:4:53,2019-01-25 13:04:53,49.409333,32.097190,88.5,91,62.0,NaN,12.119,4.6,4.6,1.81,0.1,0.2,-0.2,-1.5,-11.0
3,2019-1-25,13:4:54,2019-01-25 13:04:54,49.409300,32.097145,88.6,91,62.0,NaN,17.683,12.7,12.7,4.91,0.2,0.2,-0.3,-1.3,2.0
4,2019-1-25,13:4:55,2019-01-25 13:04:55,49.409300,32.097145,88.6,91,62.0,NaN,8.135,20.8,20.8,0.00,0.2,0.2,-0.3,-1.4,0.0


In [10]:
# WEATHER
#
# https://darksky.net/dev/docs
# '''
# UNITS
# summary: Any summaries containing temperature or snow accumulation units will have their 
#          values in degrees Celsius or in centimeters (respectively).
# nearestStormDistance: Kilometers.
# precipIntensity: Millimeters per hour.
# precipIntensityMax: Millimeters per hour.
# precipAccumulation: Centimeters.
# temperature: Degrees Celsius.
# temperatureMin: Degrees Celsius.
# temperatureMax: Degrees Celsius.
# apparentTemperature: Degrees Celsius.
# dewPoint: Degrees Celsius.
# windSpeed: Meters per second.
# windGust: Meters per second.
# pressure: Hectopascals.
# visibility: Kilometers.
# '''

# """
# Weather data parametrs

# apparentTemperature: the apparent (or “feels like”) temperature in degrees Fahrenheit.
# cloudCover: the percentage of sky occluded by clouds, between 0 and 1, inclusive.
# dewPoint optional: the dew point in degrees Fahrenheit.
# humidity optional: The relative humidity, between 0 and 1, inclusive.
# icon optional: A machine-readable text summary of this data point, suitable for selecting an icon for display. 
#               one of the following values: clear-day, clear-night, rain, snow, sleet, wind, fog, cloudy,
#               partly-cloudy-day, or partly-cloudy-night.
# ozone: the columnar density of total atmospheric ozone at the given time in Dobson units.
# precipAccumulation optional: The amount of snowfall accumulation expected to occur, in inches. 
#                              (If no snowfall is expected, this property will not be defined.)
# precipIntensity optional: The intensity (in inches of liq. water per hour) of precipitation occurring at the given time.
#                           This value is conditional on probability (that is, assuming any precipitation occurs at all).
# precipProbability optional: The probability of precipitation occurring, between 0 and 1, inclusive.
# precipType optional: The type of precipitation occurring at the given time. 
#                    If defined, this property will have one of the following values: "rain", "snow", or "sleet"tery mix”).
# pressure optional: The sea-level air pressure in millibars.
# summary optional: A human-readable text summary of this data point.
# temperature optional: The air temperature in degrees Fahrenheit.
# uvIndex optional: The UV index.
# visibility optional: The average visibility in miles, capped at 10 miles.
# windBearing optional: The direction that the wind is coming from in degrees, with true north at 0°  
#                       and progressing clockwise. (If windSpeed is zero, then this value will not be defined.)
# windGust optional: The wind gust speed in miles per hour.
# windSpeed optional: The wind speed in miles per hour.
# """

In [11]:
# Function for Weather data

def weather_metric(data):
    '''weather get api Darsky'''
    
    # Pre-define values for weather history
    #7dc0ac0a1c0e06acfb268f9e66eea991
    #f1a0a87113118f0b07a88f126e7d70e3
    
    darksky = pydarksky.DarkSky('f1a0a87113118f0b07a88f126e7d70e3')
    darksky.lang = 'English'
    darksky.units = "si"
    darksky.exclude = ['minutely', 'alerts', 'flags']
    
    # weater request
    weather = darksky.weather(longitude=data[0][2], latitude=data[0][1], date=data[0][0])
    return weather

def hourly_data_choose(data):
    '''evaluate index'''
    start_gps_time = int((data[0].to_pydatetime() - datetime.datetime(1970,1,1)).total_seconds())
         
    index_hour_24 = min( map(lambda x: (x[0], abs(x[1] - start_gps_time)), enumerate(hour_24)), key = lambda t: t[1] )[0]
    
    return index_hour_24


def weather_main(user_data):
    '''add weather data to csv file -> result pd.DataFrame'''
    global hour_24
    
    # Main gps points data for API request
    datas = user_data[['data_time','latitude','longitude']].values

    # 24 hours segments
    hourly_data = weather_metric(datas).json['hourly']['data']
    hour_24 = [hour['time'] for hour in hourly_data]
    
    # Columns for DataFrame
    columns_data = list( hourly_data[0].keys() )

    # index evaluating
    index_hour_24 = map(hourly_data_choose, datas)

    # weather evaluating by index
    weather_hourly_data = list( map(lambda x: [*x.values()], map(lambda x: hourly_data[x], index_hour_24) ) )
    
    # dataframes from weather data
    column = ['time','summary','icon','precipIntensity','precipProbability','Temperature','apparentTemperature',\
        'dewPoint','humidity','pressure','windSpeed','windGust','windBearing','cloudCover','uvIndex','visibility','ozone']
    df_weather = pd.DataFrame(np.vstack((weather_hourly_data)),columns=columns_data)

    # result dataframe
    result_user_data = pd.concat([user_data.loc[:,'data_time':],\
                                  df_weather.loc[:, 'summary':'ozone']], axis=1)
    
    return result_user_data

#user_data_weather = list( map(weather_main, user_data) )

In [12]:
%%time
# Add weather data to csv files

user_data_weather = []

for ind, (data, file) in enumerate(zip(user_data_features, file_names)):
    
    start_time = timeit.default_timer()
    
    user_data_weather.append(weather_main(data))
    
    end_time = timeit.default_timer() - start_time
    
    print(f'file: {ind + 1} -> {file}  time: {end_time:.0f} s \n')

file: 1 -> gpx\Snow_corridor_Street_porridge.gpx  time: 1 s 

file: 2 -> gpx\Snow_waist_Water_under_snow_Hard_snow.gpx  time: 1 s 

file: 3 -> gpx\Some_intervals_in_snow_Frosty_roads.gpx  time: 1 s 

Wall time: 3.22 s


In [13]:
user_data_weather[1].head(5)

,data_time,latitude,longitude,elevation,heart_rate,cadence,temperature,speed,distance_2d,distance_3d,...,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone
0,2019-01-27 13:10:00,49.409740,32.099832,85.8,85,0.0,NaN,6.543,0.0,0.0,...,-8.4,0.84,1005,5.61,8.22,314,1,0,16.09,403.83
1,2019-01-27 13:10:00,49.409756,32.099837,85.8,85,0.0,NaN,7.270,1.8,1.8,...,-8.4,0.84,1005,5.61,8.22,314,1,0,16.09,403.83
2,2019-01-27 13:10:01,49.409773,32.099853,85.9,85,81.0,NaN,4.985,5.8,5.8,...,-8.4,0.84,1005,5.61,8.22,314,1,0,16.09,403.83
3,2019-01-27 13:10:02,49.409770,32.099859,85.9,86,81.0,NaN,6.445,9.7,9.7,...,-8.4,0.84,1005,5.61,8.22,314,1,0,16.09,403.83
4,2019-01-27 13:10:03,49.409789,32.099889,85.9,86,81.0,NaN,9.286,16.5,16.5,...,-8.4,0.84,1005,5.61,8.22,314,1,0,16.09,403.83


In [14]:
user_data_weather[1].loc[0]

data_time              2019-01-27 13:10:00
latitude                           49.4097
longitude                          32.0998
elevation                             85.8
heart_rate                              85
cadence                                  0
temperature                            NaN
speed                                6.543
distance_2d                              0
distance_3d                              0
distance_points_2d                       0
uphill_gain                              0
downhill_loss                            0
grade                                  0.9
slope                                  1.3
slope_points                             0
summary                           Overcast
icon                                cloudy
precipIntensity                      0.033
precipProbability                     0.08
precipAccumulation                   0.048
precipType                            snow
temperature                          -6.08
apparentTem

In [23]:
# EXPORT to csv

# Path to data - files operations
files_csv_features = [ ('csv_weather_fetures\\' + file[:len(file) - 4] + '.csv') for file in os.listdir('csv')]
print('\n path to csv: \n\n',files_csv_features)

# export to csv files

for ind, (data, file) in enumerate(zip(user_data_weather, files_csv_features)):
    
    start_time = timeit.default_timer()
    
    data.to_csv(path_or_buf=file)
    
    end_time = timeit.default_timer() - start_time
    
    print(f'\n file: {ind + 1} -> {file}  time: {end_time:.0f} s ')


 path to csv: 

 ['csv_weather_fetures\\Snow_corridor_Street_porridge.csv', 'csv_weather_fetures\\Snow_waist_Water_under_snow_Hard_snow.csv', 'csv_weather_fetures\\Some_intervals_in_snow_Frosty_roads.csv']

 file: 1 -> csv_weather_fetures\Snow_corridor_Street_porridge.csv  time: 0 s 

 file: 2 -> csv_weather_fetures\Snow_waist_Water_under_snow_Hard_snow.csv  time: 0 s 

 file: 3 -> csv_weather_fetures\Some_intervals_in_snow_Frosty_roads.csv  time: 0 s 


In [16]:
# from pathlib import Path
# import sys

# current_dir = get_ipython().getoutput('pwd')
# module_path = Path(current_dir[0]).parent

# if module_path not in sys.path:
#     sys.path.append(str(module_path))

In [17]:
# GPX PARSE VARIANTS

# # File OPEN variants

# # 1 variant
# gpx_file = open('Some_intervals_in_snow_Frosty_roads.gpx', 'r')
# gpx_file.close()
# gpx_file

# # 2 variant
# with open('gpx\Cold_winds_Chygyryn_region.gpx','r') as gpxfile:
#     gpx = gpxpy.parse(gpxfile)
# gpx.tracks[0].segments[0].points;

# # 3 variant
# gpx_file = open( 'gpx\Cold_winds_Chygyryn_region.gpx', 'r' )
# gpx_parser = parser.GPXParser( gpx_file )
# gpx_parser.parse()
# #gpx_file.close()
# gpx_parser.gpx.tracks[0].segments[0].points[0]

In [18]:
# import gpxpy.parser as parser

# gpx_file = open( 'Some_intervals_in_snow_Frosty_roads.gpx', 'r' )

# gpx_parser = parser.GPXParser( gpx_file )
# gpx_parser.parse()

# gpx_file.close()

# gpx = gpx_parser.get_gpx()

# for track in gpx.tracks:
#     for segment in track.segments:
#         for point in segment.points:
#             print('Point at ({0},{1}) -> {2}'.format( point.latitude, point.longitude, point.elevation ))

# for waypoint in gpx.waypoints:
#     print( 'waypoint {0} -> ({1},{2})'.format( waypoint.name, waypoint.latitude, waypoint.longitude ))

# for route in gpx.routes:
#     print('Route:')
#     for point in route:
#         print( 'Point at ({0},{1}) -> {2}'.format( point.latitude, point.longitude, point.elevation ))

In [19]:
# def parsegpx():
#     points2 = list()
#     with open('Some_intervals_in_snow_Frosty_roads.gpx','r') as gpxfile:
#         gpx = gpxpy.parse(gpxfile)
#         for track in gpx.tracks:
#             for segment in track.segments:
#                 for point in segment.points:
#                     dict = {'Timestamp': point.time,
#                             'Latitude': point.latitude,
#                             'Longitude': point.longitude,
#                             'Elevation': point.elevation
#                             }
#                     points2.append(dict)
#     print(points2)

# if __name__ == '__main__':

#     parsegpx();

In [20]:
# # Weather History Data 

# # # Weather History Data API REQUEST

# # # API parameters
# # secret_key = '7dc0ac0a1c0e06acfb268f9e66eea991'
# # lat, lon = str(user_data[0].loc[0].latitude), str(user_data[0].loc[0].longitude)
# # start_time = int((user_data[0].loc[0].data_time - datetime.datetime(1970,1,1)).total_seconds())
# # #user_data[0].loc[0].data_time.to_pydatetime()

# # # Get request
# # data = requests.get(f"https://api.darksky.net/forecast/{secret_key}/{lat},{lon},{start_time}\
# # ?exclude=hourly,daily,minutely,alerts,flags?units=si").json()
# # data

# # Pre-define values
# #f1a0a87113118f0b07a88f126e7d70e3
# #7dc0ac0a1c0e06acfb268f9e66eea991
# darksky = pydarksky.DarkSky('f1a0a87113118f0b07a88f126e7d70e3')
# darksky.lang = 'Ukrainian'
# darksky.units = "si"
# darksky.exclude = ['minutely', 'alerts', 'flags']

# # weater request
# weather = darksky.weather(longitude=user_data[0].loc[0].longitude, latitude=user_data[0].loc[0].latitude,\
#                           date=user_data[0].loc[0].data_time.to_pydatetime())
# hourly_data = weather.json['hourly']['data']

# # 24 hours segments
# hour_24 = [hour['time'] for hour in weather.json['hourly']['data']]

# # pasred time
# # map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ))
# # print(datetime.datetime.utcfromtimestamp(hour_24[0]).strftime('%Y-%m-%d %H:%M:%S'))
# #actual_time = datetime.datetime.utcfromtimestamp(start_time)
# #actual_time = int((user_data[0].loc[0].data_time.to_pydatetime() - datetime.datetime(1970,1,1)).total_seconds())
# actual_time = user_data[0].loc[0].data_time.to_pydatetime()

# # difference time evaluating
# difference_time = list( map(lambda x: abs(datetime.datetime.utcfromtimestamp(x) - actual_time), hour_24) )

# # index of actual time
# index_hour_24 = difference_time.index(min(difference_time))
# print(datetime.datetime.utcfromtimestamp(hour_24[index_hour_24]).strftime('%Y-%m-%d %H:%M:%S'))


# weather.json['hourly']['data'][index_hour_24]

# [print(datetime.datetime.utcfromtimestamp(weather.hourly[i].time).strftime('%Y-%m-%d %H:%M:%S'),\
#        ' -> ',weather.hourly[i].temperature) for i in range(24)];

# print(f'\n Daily weather metrics:\n\
#     \n ozone: {weather.daily[0].ozone}\
#     \n moon Phase: {weather.daily[0].moonPhase}\
#     \n icon: {weather.daily[0].icon}\
#     \n humidity: {weather.daily[0].humidity}\
#     \n dew Point: {weather.daily[0].dewPoint}\
#     \n cloud Cover: {weather.daily[0].cloudCover}\
#     \n apparent Temperature Low: {weather.daily[0].apparentTemperatureLow}\
#     \n apparent Temperature High: {weather.daily[0].apparentTemperatureHigh}\
#     \n pressure: {weather.daily[0].pressure}\
#     \n uv Index: {weather.daily[0].uvIndex}\
#     \n wind Speed: {weather.daily[0].windSpeed}\
#     \n wind Gust: {weather.daily[0].windGust}\
#     \n wind Bearing: {weather.daily[0].windBearing}\
#     \n wind Gust: {weather.daily[0].windGust}\
#     \n wind Gust: {weather.daily[0].windGust}\
#     \n visibility: {weather.daily[0].visibility}\
#     \n summary: {weather.daily[0].summary}')

In [21]:
# FIT CONVERT

# import csv
# import os
# #to install fitparse, run 
# #sudo pip3 install -e git+https://github.com/dtcooper/python-fitparse#egg=python-fitparse
# import fitparse
# import pytz

# allowed_fields = ['timestamp','position_lat','position_long', 'distance',
# 'enhanced_altitude', 'altitude','enhanced_speed',
#                  'speed', 'heart_rate','cadence','fractional_cadence']
# required_fields = ['timestamp', 'position_lat', 'position_long', 'altitude']

# UTC = pytz.UTC
# CST = pytz.timezone('US/Central')


# def main():
#     files = os.listdir()
#     fit_files = [file for file in files if file[-4:].lower()=='.fit']
#     for file in fit_files:
#         new_filename = file[:-4] + '.csv'
#         if os.path.exists(new_filename):
#             #print('%s already exists. skipping.' % new_filename)
#             continue
#         fitfile = fitparse.FitFile(file,  
#             data_processor=fitparse.StandardUnitsDataProcessor())
        
#         print('converting %s' % file)
#         write_fitfile_to_csv(fitfile, new_filename)
#     print('finished conversions')


# def write_fitfile_to_csv(fitfile, output_file='test_output.csv'):
#     messages = fitfile.messages
#     data = []
#     for m in messages:
#         skip=False
#         if not hasattr(m, 'fields'):
#             continue
#         fields = m.fields
#         #check for important data types
#         mdata = {}
#         for field in fields:
#             if field.name in allowed_fields:
#                 if field.name=='timestamp':
#                     mdata[field.name] = UTC.localize(field.value).astimezone(CST)
#                 else:
#                     mdata[field.name] = field.value
#         for rf in required_fields:
#             if rf not in mdata:
#                 skip=True
#         if not skip:
#             data.append(mdata)
#     #write to csv
#     with open(output_file, 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(allowed_fields)
#         for entry in data:
#             writer.writerow([ str(entry.get(k, '')) for k in allowed_fields])
#     print('wrote %s' % output_file)

# if __name__=='__main__':
#     main()